In [ ]:
%cd '..'

In [ ]:
import pandas as pd
from Bio import Entrez, SeqIO
from curated_set_services import CuratedSet

In [ ]:
from Bio import Entrez, SeqIO

Entrez.email = "test@test.com"

In [ ]:
curated_set = CuratedSet()
cs = curated_set

cs.data.shape, cs.data.columns

In [ ]:
lst = [line for line in cs.data["accession"].to_list() if not "HISTDB" in line]

In [ ]:
len(lst)

In [ ]:
others_accessions = [line for line in cs.data["accession"].tolist() if "HISTDB" in line]
sup_dict = dict()
for acsn in others_accessions:
    sup_dict[acsn] = ""

In [ ]:
len(sup_dict)

In [ ]:
def Function(my_list):
    res_dict = dict()
    for acc in my_list:
        res_dict[acc] = ""  #Если GeneId не будет найден, добавляется пустая строка
        handle = Entrez.efetch(db="protein", id=acc, rettype="gb", retmode="text")
        record = SeqIO.read(handle, "genbank")
        for feature in record.features:
            if feature.type == "CDS":
                if "db_xref" in feature.qualifiers:
                    L = [
                        line
                        for line in feature.qualifiers["db_xref"]
                        if "GeneID" in line
                    ]
                    if L:
                        for i in L:
                            res_dict[acc] = i.split(":")[1]
    return res_dict

In [ ]:
main_dict = Function(lst)

In [ ]:
len(main_dict)

In [ ]:
len(lst)

In [ ]:
merged_dict = {**sup_dict, **main_dict}

In [ ]:
len(merged_dict)

In [ ]:
column = cs.data["accession"].replace(merged_dict)

In [ ]:
column.shape

In [ ]:
cs.data.ncbi_gene_id = column

In [ ]:
cs.save()